In [0]:
#%pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import *
from pyspark.ml.regression import LinearRegression , DecisionTreeRegressor, RandomForestRegressor, GBTRegressionModel, GBTRegressor

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.linalg import Vectors

from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.context import SparkContext
from pyspark.mllib.evaluation import MulticlassMetrics


In [0]:
#%pyspark
#PYSPARK_CLI = True
#if PYSPARK_CLI:
	#sc = SparkContext.getOrCreate()
	#spark = SparkSession(sc)

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2016_CommunityCollegeDistrict.csv"
#file_location = "/user/jdang8/2016_CommunityCollegeDistrict.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
#%pyspark
#df.printSchema()

In [0]:
# Create a view or table

temp_table_name = "2016_CommunityCollegeDistrict_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:

dftemp=df.select('EmployerName','Position','TotalWages', 'RegularPay', 'MaxPositionSalary', 'MinPositionSalary')
dftemp.show()

+--------------------+--------------------+----------+----------+-----------------+-----------------+
|        EmployerName|            Position|TotalWages|RegularPay|MaxPositionSalary|MinPositionSalary|
+--------------------+--------------------+----------+----------+-----------------+-----------------+
|Allan Hancock Joi...|Administrative Se...|     65114|     64191|            62460|            51384|
|Allan Hancock Joi...|Assoc Supt/Vice P...|    156257|    155952|           161721|           121867|
|Allan Hancock Joi...|Coord,Acdm Affrs/...|     54240|     52379|            66468|            54684|
|Allan Hancock Joi...|Coord,Acdm Affrs/...|     35366|     33520|            66468|            54684|
|Allan Hancock Joi...|Dean, Academic Af...|     17745|     17745|           133098|           111526|
|Allan Hancock Joi...|Dean,Academic Aff...|     65101|     62045|           129222|           108278|
|Allan Hancock Joi...|Dean,Academic Aff...|    144136|    137876|           129222

In [0]:
%sql
select * from `2016_CommunityCollegeDistrict_csv` 

In [0]:
df.take(10)

Out[8]: [Row(Year=2016, EmployerType='Community College District', EmployerName='Allan Hancock Joint Community College District', DepartmentOrSubdivision='Academic Affairs', Position='Administrative Secretary V', ElectedOfficial=False, Judicial=False, OtherPositions=None, MinPositionSalary=51384, MaxPositionSalary=62460, ReportedBaseWage=None, RegularPay=64191, OvertimePay=None, LumpSumPay=None, OtherPay=923, TotalWages=65114, DefinedBenefitPlanContribution=8539.0, EmployeesRetirementCostCovered=None, DeferredCompensationPlan=None, HealthDentalVision=5376, TotalRetirementAndHealthContribution=13915.0, PensionFormula='2% @ 55', EmployerURL='http://www.hancockcollege.edu', EmployerPopulation=None, LastUpdatedDate='08/22/2017', EmployerCounty='Santa Barbara', SpecialDistrictActivities=None, IncludesUnfundedLiability=None, SpecialDistrictType=None),
 Row(Year=2016, EmployerType='Community College District', EmployerName='Allan Hancock Joint Community College District', DepartmentOrSubdivis

In [0]:
df_sql = spark.sql("select * from 2016_CommunityCollegeDistrict_csv LIMIT 10")
display(df_sql) 

Year,EmployerType,EmployerName,DepartmentOrSubdivision,Position,ElectedOfficial,Judicial,OtherPositions,MinPositionSalary,MaxPositionSalary,ReportedBaseWage,RegularPay,OvertimePay,LumpSumPay,OtherPay,TotalWages,DefinedBenefitPlanContribution,EmployeesRetirementCostCovered,DeferredCompensationPlan,HealthDentalVision,TotalRetirementAndHealthContribution,PensionFormula,EmployerURL,EmployerPopulation,LastUpdatedDate,EmployerCounty,SpecialDistrictActivities,IncludesUnfundedLiability,SpecialDistrictType
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,Administrative Secretary V,false,false,null,51384,62460,null,64191,null,null,923,65114,8539.0,null,null,5376,13915.0,2% @ 55,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Assoc Supt/Vice Pres, Academic Affairs",false,false,null,121867,161721,null,155952,null,null,305,156257,19181.0,null,null,8076,27257.0,2% @ 60,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Coord,Acdm Affrs/Adm Sec V",false,false,null,54684,66468,null,52379,1861,null,null,54240,7714.0,null,null,14114,21828.0,2% @ 55,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Coord,Acdm Affrs/Adm Sec V",false,false,null,54684,66468,null,33520,null,null,1846,35366,4171.0,null,null,2240,6411.0,2% @ 55,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Dean, Academic Affairs",false,false,null,111526,133098,null,17745,null,null,null,17745,1986.0,null,null,1906,3892.0,2% @ 60,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Dean,Academic Affairs",false,false,null,108278,129222,null,62045,null,null,3056,65101,8670.0,null,null,6671,15341.0,2% @ 60,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Dean,Academic Affairs",false,false,null,108278,129222,null,137876,null,null,6260,144136,16104.0,null,null,null,16104.0,2% @ 60,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Dean,Academic Affairs",false,false,null,108278,129222,null,109305,null,null,22800,132105,16969.0,null,null,8076,25045.0,2% @ 60,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,"Interim Dean, Academit Affairs",false,false,null,108278,129222,null,null,null,null,null,0,-683.0,null,null,null,-683.0,null,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null
2016,Community College District,Allan Hancock Joint Community College District,Academic Affairs,Office Service Tech I,false,false,null,null,null,null,null,null,null,187,187,null,null,null,null,0.0,null,http://www.hancockcollege.edu,null,08/22/2017,Santa Barbara,null,null,null


In [0]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- EmployerType: string (nullable = true)
 |-- EmployerName: string (nullable = true)
 |-- DepartmentOrSubdivision: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- ElectedOfficial: boolean (nullable = true)
 |-- Judicial: boolean (nullable = true)
 |-- OtherPositions: string (nullable = true)
 |-- MinPositionSalary: integer (nullable = true)
 |-- MaxPositionSalary: integer (nullable = true)
 |-- ReportedBaseWage: string (nullable = true)
 |-- RegularPay: integer (nullable = true)
 |-- OvertimePay: integer (nullable = true)
 |-- LumpSumPay: integer (nullable = true)
 |-- OtherPay: integer (nullable = true)
 |-- TotalWages: integer (nullable = true)
 |-- DefinedBenefitPlanContribution: double (nullable = true)
 |-- EmployeesRetirementCostCovered: integer (nullable = true)
 |-- DeferredCompensationPlan: integer (nullable = true)
 |-- HealthDentalVision: integer (nullable = true)
 |-- TotalRetirementAndHealthContribution: do

In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- EmployerType: string (nullable = true)
 |-- EmployerName: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- MinPositionSalary: integer (nullable = true)
 |-- MaxPositionSalary: integer (nullable = true)
 |-- ReportedBaseWage: string (nullable = true)
 |-- RegularPay: integer (nullable = true)
 |-- OvertimePay: integer (nullable = true)
 |-- LumpSumPay: integer (nullable = true)
 |-- OtherPay: integer (nullable = true)
 |-- TotalWages: integer (nullable = true)
 |-- LastUpdatedDate: string (nullable = true)
 |-- EmployerCounty: string (nullable = true)



In [0]:
display(df2)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|Year|        EmployerType|        EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|
+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|2016|Community College...|Allan Hancock Joi...|Administrative Se...|            51384|            62460|              NA|     64191|          0|         0|     923|     65114|     08/22/2017| Santa Barbara|
|2016|Community College...|Allan Hancock Joi...|Assoc Supt/Vice P...|           121867|           161721|              NA|    155952|          0|         0|     305|   

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

Training Rows: 113110 Testing Rows: 48753


In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|Year|        EmployerType|        EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|               wages|
+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|2016|Community College...|Allan Hancock Joi...|Academic Affairs ...|            37572|            45672|              NA|     39130|       3224|         0|    1215|     43569|     08/22/2017| Santa Barbara|[39130.0,43569.0,...|
|2016|Community College...|Allan Hancock Joi...|Academic Load/Sch...|               

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

,trueWages,importance
0,RegularPay,0.458570
1,TotalWages,0.388816
3,MinPositionSalary,0.125536
2,MaxPositionSalary,0.027078


In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[29753.0,30676.0,...|30450.668445059204|     30676|
|[37806.0,41243.0,...|40853.351502020305|     41243|
|[38434.0,41029.0,...|40853.351502020305|     41029|
|[38605.0,40697.0,...|40853.351502020305|     40697|
|[41407.0,42886.0,...|40853.351502020305|     42886|
|[50597.0,51520.0,...| 53924.53564709512|     51520|
|[28418.0,29666.0,...|28734.401883139933|     29666|
|[9059.0,9387.0,37...|10271.921654551828|      9387|
|[41029.0,41498.0,...|40853.351502020305|     41498|
|   (4,[1],[11577.0])|   9732.9283547194|     11577|
|[126093.0,127591....| 132908.4876101362|    127591|
|[103967.0,104813....|109716.46644571362|    104813|
|[36846.0,39306.0,...| 40699.14930525991|     39306|
|[37022.0,48104.0,...| 49554.84789048635|     48104|
|   (4,[1],[12276.0])|   9732.9283547194|     12276|
|[7012.0,7790.0,40...|  8302.20843677151|     

In [0]:
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

R Squared (R2) on test data = 0.985436
RMSE: 4762.131585


#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|Year|        EmployerType|        EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|
+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|2016|Community College...|Allan Hancock Joi...|Administrative Se...|            51384|            62460|              NA|     64191|          0|         0|     923|     65114|     08/22/2017| Santa Barbara|
|2016|Community College...|Allan Hancock Joi...|Assoc Supt/Vice P...|           121867|           161721|              NA|    155952|          0|         0|     305|   

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

Training Rows: 113503 Testing Rows: 48360


In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|Year|        EmployerType|        EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|               wages|
+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+--------------------+
|2016|Community College...|Allan Hancock Joi...|Academic Affairs ...|            37572|            45672|              NA|     39130|       3224|         0|    1215|     43569|     08/22/2017| Santa Barbara|[39130.0,43569.0,...|
|2016|Community College...|Allan Hancock Joi...|Academic Resource...|               

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#@%pyspark
test.show()


+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|Year|        EmployerType|        EmployerName|            Position|MinPositionSalary|MaxPositionSalary|ReportedBaseWage|RegularPay|OvertimePay|LumpSumPay|OtherPay|TotalWages|LastUpdatedDate|EmployerCounty|
+----+--------------------+--------------------+--------------------+-----------------+-----------------+----------------+----------+-----------+----------+--------+----------+---------------+--------------+
|2016|Community College...|Allan Hancock Joi...|Academic Load/Sch...|                0|                0|              NA|     70742|      25329|         0|     305|     96376|     08/22/2017| Santa Barbara|
|2016|Community College...|Allan Hancock Joi...|Academic Support ...|            36576|            44448|              NA|     26001|          0|         0|     749|   

In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[70742.0,96376.0,...|  86765.9053490967|     96376|
|[26001.0,26750.0,...| 27545.76141806553|     26750|
|[33454.0,34377.0,...| 32945.10202846823|     34377|
|[41949.0,42919.0,...|42322.724655318816|     42919|
|[50242.0,51605.0,...| 54490.40941587076|     51605|
|[30856.0,35810.0,...| 32945.10202846823|     35810|
|[45547.0,47342.0,...| 46697.49318634725|     47342|
|[9059.0,9387.0,37...| 9342.930970977503|      9387|
|[39146.0,39503.0,...| 39332.67473338529|     39503|
|[45055.0,45360.0,...| 46697.49318634725|     45360|
|   (4,[1],[11577.0])| 11698.39009322398|     11577|
|[144199.0,145676....|167339.91134702042|    145676|
|[37022.0,48104.0,...| 46697.49318634725|     48104|
|[61825.0,74851.0,...| 73041.68907917959|     74851|
|[55197.0,62482.0,...|62535.293972180814|     62482|
|[7308.0,7308.0,40...|  6817.98202386153|     

In [0]:
#%pyspark
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[70742.0,96376.0,...|  86765.9053490967|     96376|
|[26001.0,26750.0,...| 27545.76141806553|     26750|
|[33454.0,34377.0,...| 32945.10202846823|     34377|
|[41949.0,42919.0,...|42322.724655318816|     42919|
|[50242.0,51605.0,...| 54490.40941587076|     51605|
|[30856.0,35810.0,...| 32945.10202846823|     35810|
|[45547.0,47342.0,...| 46697.49318634725|     47342|
|[9059.0,9387.0,37...| 9342.930970977503|      9387|
|[39146.0,39503.0,...| 39332.67473338529|     39503|
|[45055.0,45360.0,...| 46697.49318634725|     45360|
|   (4,[1],[11577.0])| 11698.39009322398|     11577|
|[144199.0,145676....|167339.91134702042|    145676|
|[37022.0,48104.0,...| 46697.49318634725|     48104|
|[61825.0,74851.0,...| 73041.68907917959|     74851|
|[55197.0,62482.0,...|62535.293972180814|     62482|
|[7308.0,7308.0,40...|  6817.98202386153|     

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

R Squared (R2) on test data = 0.986276
RMSE: 4631.387430


In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

+--------------------+------------------+----------+
|               wages|        prediction|TotalWages|
+--------------------+------------------+----------+
|[70742.0,96376.0,...|  86765.9053490967|     96376|
|[26001.0,26750.0,...| 27545.76141806553|     26750|
|[33454.0,34377.0,...| 32945.10202846823|     34377|
|[41949.0,42919.0,...|42322.724655318816|     42919|
|[50242.0,51605.0,...| 54490.40941587076|     51605|
|[30856.0,35810.0,...| 32945.10202846823|     35810|
|[45547.0,47342.0,...| 46697.49318634725|     47342|
|[9059.0,9387.0,37...| 9342.930970977503|      9387|
|[39146.0,39503.0,...| 39332.67473338529|     39503|
|[45055.0,45360.0,...| 46697.49318634725|     45360|
|   (4,[1],[11577.0])| 11698.39009322398|     11577|
|[144199.0,145676....|167339.91134702042|    145676|
|[37022.0,48104.0,...| 46697.49318634725|     48104|
|[61825.0,74851.0,...| 73041.68907917959|     74851|
|[55197.0,62482.0,...|62535.293972180814|     62482|
|[7308.0,7308.0,40...|  6817.98202386153|     

In [0]:
####CODE BREAK for 2015

#CC2015

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2015_CommunityCollegeDistrict.csv"
#file_location = "/user/jdang8/2015_CommunityCollegeDistrict.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2015_CommunityCollegeDistrict_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

root



In [0]:
display(df2)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

++
||
++
++



In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

Training Rows: 0 Testing Rows: 0


In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-1679082689280255> in <module>
----> 1 training = assembler.transform(train)
      2 training.show()

/databricks/spark/python/pyspark/ml/base.py in transform(self, dataset, params)
    215                 return self.copy(params)._transform(dataset)
    216             else:
--> 217                 return self._transform(dataset)
    218         else:
    219             raise TypeError("Params must be a param map but got %s." % type(params))

/databricks/spark/python/pyspark/ml/wrapper.py in _transform(self, dataset)
    348     def _transform(self, dataset):
    349         self._transfer_params_to_java()
--> 350         return DataFrame(self._java_obj.transform(dataset._jdf), dataset.sql_ctx)
    351 
    352 

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303  

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

In [0]:
#GBT 2015


#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#### Code Break for 2017

#CC 2017

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2017_CommunityCollegeDistrict.csv"
#file_location = "/user/jdang8/2017_CommunityCollegeDistrict.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2017_CommunityCollegeDistrict_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#@%pyspark
test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
# Code Break for 2018

#CC 2018

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2018_CommunityCollegeDistrict.csv"
#file_location = "/user/jdang8/2018_CommunityCollegeDistrict.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2018_CommunityCollegeDistrict_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

In [0]:
#@%pyspark
test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
### Code Break for 2019

#CC 2019

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2019_CommunityCollegeDistrict.csv"
#file_location = "/user/jdang8/2019_CommunityCollegeDistrict.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2019_CommunityCollegeDistrict_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)

In [0]:
#%pyspark
model = cv.fit(train)

test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
### Code break for 2020

#CC 2020

In [0]:
#%pyspark
# File location and type
file_location = "/FileStore/tables/2020_CommunityCollegeDistrict.csv"
#file_location = "/user/jdang8/2020_CommunityCollegeDistrict.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type).option("inferSchema", infer_schema)\
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "2020_CommunityCollegeDistrict_csv"

df.createOrReplaceTempView(temp_table_name)


In [0]:
df2 = df.drop("DepartmentOrSubdivision", "ElectedOfficial", "Judicial","OtherPositions", "DefinedBenefitPlanContribution","EmployeesRetirementCostCovered", "DeferredCompensationPlan","HealthDentalVision", "TotalRetirementAndHealthContribution","PensionFormula", "EmployerURL","EmployerPopulation","IncludesUnfundedLiability", "SpecialDistrictActivities"," IncludesUnfundedLiability","SpecialDistrictType")
df2.printSchema()

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
training = assembler.transform(train)
training.show()

#Random Forest Regression

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages", featuresCol="wages", numTrees=10, maxDepth=5)


In [0]:
pipeline = Pipeline(stages=[assembler, rf])

In [0]:
model = pipeline.fit(train)

In [0]:
rfModel = model.stages[-1]
print(rfModel.toDebugString)

In [0]:
#importance column name can be changed

import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(),
rfModel.featureImportances)),
columns=["trueWages", "importance"])
featureImp.sort_values(by="importance", ascending=False)

In [0]:
rf = RandomForestRegressor(labelCol="TotalWages",featuresCol="wages",
numTrees=10, maxDepth=5)
# Combine stages into pipeline
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train)

In [0]:
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()
display(predicted)

In [0]:
    
      from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="TotalWages",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(prediction))
rf_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % rf_evaluator.evaluate(prediction))

#Gradient Boost Tree (GBT)

In [0]:
#%pyspark
df3 = df2.na.fill(value=0).na.fill("NA")
df3.show()

In [0]:
#%pyspark
splits = df3.randomSplit([0.7,0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, "Testing Rows:", test_rows)

In [0]:
#%pyspark
assembler = VectorAssembler(inputCols = ["RegularPay", "TotalWages","MaxPositionSalary", "MinPositionSalary"], outputCol="wages")

In [0]:
#%pyspark
training = assembler.transform(train)
training.show()

In [0]:
#%pyspark

gbt = GBTRegressor(labelCol="TotalWages", featuresCol="wages", maxDepth=5)

In [0]:
#%pyspark
paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxDepth, [2, 5])\
.addGrid(gbt.maxIter, [10, 20])\
.build()

In [0]:
#%pyspark
#gbt_evaluator = RegressionEvaluator(predictionCol="prediction", \ labelCol="TotalWages",metricName="r2")

gbt_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="TotalWages", metricName="r2")


In [0]:
#%pyspark
pipeline = Pipeline(stages=[assembler, gbt])

In [0]:
#%pyspark
cv = CrossValidator(estimator=pipeline, evaluator= gbt_evaluator, estimatorParamMaps=paramGrid)
#%pyspark
model = cv.fit(train)
#@%pyspark
test.show()


In [0]:
#%pyspark
prediction = model.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()

In [0]:
#%pyspark
print("R Squared (R2) on test data = %g" % gbt_evaluator.evaluate(prediction))
gbt_evaluator = RegressionEvaluator(labelCol="TotalWages", predictionCol="prediction", metricName="rmse")
print("RMSE: %f" % gbt_evaluator.evaluate(prediction))

In [0]:
#%pyspark
cva = TrainValidationSplit(estimator= pipeline, evaluator=gbt_evaluator, estimatorParamMaps=paramGrid, trainRatio=0.8)


example = cva.fit(train)

In [0]:

# Transform the test data and generate predictions by applying the trained model
prediction = example.transform(test)
predicted = prediction.select("wages", "prediction", "TotalWages")
predicted.show()